# The Revenge of Rocchio's Angels

We will be #1 this time

Local Script Dependencies

In [ ]:
!python -c "import platform; print(platform.platform()); import sys; print(sys.version)"

In [1]:
from engine import SearchEngine
from evaluate_map import *
from optimizing import Optimize

C:\Users\ariel\PycharmProjects\FinalProject\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
se = SearchEngine()

In [4]:
se.set_searcher(approach="qld")

In [ ]:
results = se.get_top_k("international organized crime", k=50)

In [ ]:
results[0][0][:300]

In [2]:
topics = load_topics("Data/queriesROBUST.txt")

In [6]:
# This one creates a file called "run.txt" with submitting format, can change file name
se.search_all_queries(topics, 100)

In [7]:
qrels = load_qrels("Data/qrels_50_Queries")   # or "qrel301.txt"
run   = load_run("run.txt")

map_score, ap_by_q = mean_average_precision(qrels, run)
map_score

0.24594939993995996

In [7]:
map = get_map_by_paths("Data/qrels_50_Queries", "run.txt")

In [8]:
map

0.21283191880598026

In [3]:
opti = Optimize()
fb_terms_values = [5, 6, 8, 10, 15, 20]
fb_docs_values = [5, 7, 10, 15]
og_query_weight_values = [0.3, 0.4, 0.5, 0.6, 0.7]

opti.optimize_qld(topics, fb_terms_values,fb_docs_values,og_query_weight_values, k=1000)

fb_terms=5, fb_docs=5, w=0.3 -> MAP=0.206536
fb_terms=5, fb_docs=5, w=0.4 -> MAP=0.219894
fb_terms=5, fb_docs=5, w=0.5 -> MAP=0.234482
fb_terms=5, fb_docs=5, w=0.6 -> MAP=0.241556
fb_terms=5, fb_docs=5, w=0.7 -> MAP=0.245587
fb_terms=5, fb_docs=7, w=0.3 -> MAP=0.193709
fb_terms=5, fb_docs=7, w=0.4 -> MAP=0.210535
fb_terms=5, fb_docs=7, w=0.5 -> MAP=0.224581
fb_terms=5, fb_docs=7, w=0.6 -> MAP=0.232520
fb_terms=5, fb_docs=7, w=0.7 -> MAP=0.240887
fb_terms=5, fb_docs=10, w=0.3 -> MAP=0.208316
fb_terms=5, fb_docs=10, w=0.4 -> MAP=0.225750
fb_terms=5, fb_docs=10, w=0.5 -> MAP=0.235818
fb_terms=5, fb_docs=10, w=0.6 -> MAP=0.244223
fb_terms=5, fb_docs=10, w=0.7 -> MAP=0.246381
fb_terms=5, fb_docs=15, w=0.3 -> MAP=0.216002
fb_terms=5, fb_docs=15, w=0.4 -> MAP=0.230370
fb_terms=5, fb_docs=15, w=0.5 -> MAP=0.241824
fb_terms=5, fb_docs=15, w=0.6 -> MAP=0.247486
fb_terms=5, fb_docs=15, w=0.7 -> MAP=0.250858
fb_terms=6, fb_docs=5, w=0.3 -> MAP=0.212900
fb_terms=6, fb_docs=5, w=0.4 -> MAP=0.226348
